In [ ]:
import pandas as pd
import os
import random

In [ ]:
# Get paths to working directory and files folder
dir_path = os.path.dirname(os.path.abspath(os.curdir))
dir_path = os.path.join(dir_path, 'Mutational_Signatures')
print(dir_path)

In [ ]:
# Read in the tables
cosmic_signatures_path = os.path.join(dir_path, 'cosmic_signatures')

df_4 = pd.read_csv(os.path.join(cosmic_signatures_path, 'SBS4.csv'))
df_6 = pd.read_csv(os.path.join(cosmic_signatures_path, 'SBS6.csv'))
df_7a = pd.read_csv(os.path.join(cosmic_signatures_path, 'SBS7a.csv'))
df_9 = pd.read_csv(os.path.join(cosmic_signatures_path, 'SBS9.csv'))
df_18 = pd.read_csv(os.path.join(cosmic_signatures_path, 'SBS18.csv'))

print(df_7a.to_string)

In [ ]:
df_grch37 = pd.DataFrame().assign(SBS4=df_4['SBS4_GRCh37'], SBS6=df_6['SBS6_GRCh37'], SBS7a=df_7a['SBS7a_GRCh37'], SBS9=df_9['SBS9_GRCh37'], SBS18=df_18['SBS18_GRCh37'])
df_grch37

In [ ]:
df_sparse = pd.DataFrame()
for i in range(50):
    df_sparse[i] = [random.random()*1.5+0.5 if random.uniform(0, 1) > 0.6 else 0 for x in range(0, df_grch37.shape[1])]
df_sparse = df_sparse.set_index(df_grch37.columns)
df_sparse

In [ ]:
simulated_data = df_grch37.dot(df_sparse)
simulated_data